In [1]:
import os
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
path = './10-Scene/'
image_path = []
for i in range(10):
    dir = os.path.join(path, str(i))
    for file in os.listdir(dir):
        image_path.append(os.path.join(dir, file))

In [3]:
def bag_of_features(features, centres, k=500):
    vec = np.zeros((1, k))
    for i in range(features.shape[0]):
        feat = features[i]
        diff = np.tile(feat, (k, 1)) - centres
        dist = pow(((pow(diff, 2)).sum(axis = 1)), 0.5)
        idx_dist = dist.argsort()
        idx = idx_dist[0]
        vec[0][idx] += 1
    return vec

In [4]:
def CalcFeatures(img):
        sift = cv2.SIFT_create()   
        kp, des = sift.detectAndCompute(img, None)
        return des

In [5]:
def SIFT_Kmean(k):

    features = []
    for file in image_path:
        img = cv2.imread(file, 0)
        img_des = CalcFeatures(img)
        if img_des is not None:
            features.append(img_des)
    features = np.vstack(features)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 0.1)
    flags = cv2.KMEANS_RANDOM_CENTERS
    compactness, labels, centres = cv2.kmeans(features, k, None, criteria, 10, flags) #Kmean的参数也可调

    labels = []
    vec = []
    for file in image_path:
        img = cv2.imread(file, 0)
        img_des = CalcFeatures(img)
        if img_des is not None:
            img_vec = bag_of_features(img_des, centres, k)
            vec.append(img_vec)
            temp = os.path.split(file)[0]
            temp1 = os.path.split(temp)[1]
            labels.append(int(temp1))
    vec = np.vstack(vec)
    
    return vec, labels

In [6]:
K = [100,200,300,400,500,600,700,800,900,1000] #100 - 1000
for k in K:
    image_vec, image_labels = SIFT_Kmean(k)
    np.save('image_vec_'+str(k)+'.npy', image_vec)
    np.save('image_label_'+str(k)+'.npy',image_labels)